## PyTorch 선형 회귀 구현

In [1]:
import pandas as pd
import numpy as np 
import torch 

import matplotlib.pyplot as plt 
%matplotlib inline

In [17]:
torch.manual_seed(7777)

In [17]:
data_url = "http://lib.stat.cmu.edu/datasets/boston"
raw_df = pd.read_csv(data_url, sep="\s+", skiprows=22, header=None)
data = np.hstack([raw_df.values[::2, :], raw_df.values[1::2, :2]])
target = raw_df.values[1::2, 2]

#columns = ['crim', 'zn', 'indus', 'chas', 'nox', 'rm', 'age', 'dis', 'rad', 'tax', 'ptratio', 'black', 'lstat']
columns = ['CRIM', 'ZN', 'INDUS', 'CHAS', 'NOX', 'RM', 'AGE', 'DIS', 'RAD', 'TAX', 'PTRATIO', 'B', 'LSTAT']
df = pd.DataFrame(data,columns=columns)
#df['PRICE'] = target
df['const'] = np.ones(df.shape[0])
df.tail()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,const
501,0.06263,0.0,11.93,0.0,0.573,6.593,69.1,2.4786,1.0,273.0,21.0,391.99,9.67,1.0
502,0.04527,0.0,11.93,0.0,0.573,6.120,76.7,2.2875,1.0,273.0,21.0,396.90,9.08,1.0
503,0.06076,0.0,11.93,0.0,0.573,6.976,91.0,2.1675,1.0,273.0,21.0,396.90,5.64,1.0
504,0.10959,0.0,11.93,0.0,0.573,6.794,89.3,2.3889,1.0,273.0,21.0,393.45,6.48,1.0
505,0.04741,0.0,11.93,0.0,0.573,6.030,80.8,2.5050,1.0,273.0,21.0,396.90,7.88,1.0


In [4]:
#### 저작권 이슈로 사용이 불가하다고 한다. ㅠㅠ
from sklearn.datasets import load_boston
boston = load_boston()
df = pd.DataFrame(boston.data, columns=boston.feature_names)
df['const'] = np.ones(df.shape[0])
df.tail()

/usr/local/lib/python3.9/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function load_boston is deprecated; `load_boston` is deprecated in 1.0 and will be removed in 1.2.

    The Boston housing prices dataset has an ethical problem. You can refer to
    the documentation of this function for further details.

    The scikit-learn maintainers therefore strongly discourage the use of this
    dataset unless the purpose of the code is to study and educate about
    ethical issues in data science and machine learning.

    In this special case, you can fetch the dataset from the original
    source::

        import pandas as pd
        import numpy as np


        data_url = "http://lib.stat.cmu.edu/datasets/boston"
        raw_df = pd.read_csv(data_url, sep="\s+", skiprows=22, header=None)
        data = np.hstack([raw_df.values[::2, :], raw_df.values[1::2, :2]])
        target = raw_df.values[1::2, 2]

    Alternative datasets include the California housing dataset (i.e.

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,const
501,0.06263,0.0,11.93,0.0,0.573,6.593,69.1,2.4786,1.0,273.0,21.0,391.99,9.67,1.0
502,0.04527,0.0,11.93,0.0,0.573,6.120,76.7,2.2875,1.0,273.0,21.0,396.90,9.08,1.0
503,0.06076,0.0,11.93,0.0,0.573,6.976,91.0,2.1675,1.0,273.0,21.0,396.90,5.64,1.0
504,0.10959,0.0,11.93,0.0,0.573,6.794,89.3,2.3889,1.0,273.0,21.0,393.45,6.48,1.0
505,0.04741,0.0,11.93,0.0,0.573,6.030,80.8,2.5050,1.0,273.0,21.0,396.90,7.88,1.0


#### 수학적 방식 OLS???

In [18]:
n, m = df.shape
df.shape

(506, 14)

In [19]:
x = torch.tensor(df.values)
y = torch.tensor(target).view(-1, 1)

x.shape, y.shape

(torch.Size([506, 14]), torch.Size([506, 1]))

In [20]:
XT = torch.transpose(x, 0, 1)
XT.shape

torch.Size([14, 506])

In [21]:
w = torch.matmul(torch.matmul(torch.inverse(torch.matmul(XT, x)), XT), y)
y_pred = torch.matmul(x, w)

print("예측한 집값 :", y_pred[19], "실제 집값 :", target[19])
print("예측한 집값 :", y_pred[10], "실제 집값 :", target[10])

예측한 집값 : tensor([18.4061], dtype=torch.float64) 실제 집값 : 18.2
예측한 집값 : tensor([18.9995], dtype=torch.float64) 실제 집값 : 15.0


#### Gradient descent 방식

In [22]:
w = torch.rand((14, 1), dtype=torch.float64, requires_grad=True)
b = torch.rand((1, 1),  dtype=torch.float64, requires_grad=True)

In [23]:
z = x.matmul(w) + b
loss = torch.mean((z - y)**2)  

In [24]:
w.grad, b.grad

(None, None)

In [25]:
loss.backward()

In [26]:
w.grad, b.grad

(tensor([[4.2991e+03],
         [6.7920e+03],
         [1.0170e+04],
         [5.1937e+01],
         [4.5796e+02],
         [4.8648e+03],
         [5.8423e+04],
         [2.6462e+03],
         [9.6833e+03],
         [3.6684e+05],
         [1.4768e+04],
         [2.6972e+05],
         [1.1155e+04],
         [7.8464e+02]], dtype=torch.float64),
 tensor([[784.6375]], dtype=torch.float64))

In [27]:
print(loss)

tensor(173420.1276, dtype=torch.float64, grad_fn=<MeanBackward0>)


In [31]:
# requires_grad = True 일경우 numpy 호출이 불가능
loss.numpy()

RuntimeError: Can't call numpy() on Tensor that requires grad. Use tensor.detach().numpy() instead.

requires_grad를 없애고 값을 불러와야한다.

In [29]:
loss.detach().numpy() # 값 value만 가져오기

array(173420.12761429)

In [30]:
with torch.no_grad(): # 값 value만 가져오기
    print(loss.numpy())

173420.12761428562


pytorch에서 그냥 item으로 바로 값을 추출 가능하다....

In [32]:
loss.item()

173420.12761428562

#### 1. backward()로 학습
#### assign 대신에 data에 접근해서 값을 수정 

tensor.data = 다른데이터

In [33]:
learning_rate = 0.000003 # 3e-7

for epoch in range(100):
    # 위에 데이터프레임에 const = 1로 상수값을 줘서 사실 b는 필요가없다.
    # 강의 내용상 b를 포함해서 진행
    z = x.matmul(w) + b
    loss = torch.mean((y - z)**2)  
    
    # 미분 계산
    loss.backward()
    
    # 가중치 업데이트
    w.data -= learning_rate * w.grad
    b.data -= learning_rate * b.grad
    
    # 진행상황 출력
    print("{:3} - loss : {}".format(epoch, loss.item()), end="\r")
    
    # w.grad의 grad는 누적해서 구하게 된다. 
    # 그래서 epoch가 끝나면 초기화를 해줘야한다.
    w.grad.zero_()
    b.grad.zero_()

#### 2. torch.autograd.grad 방식

In [34]:
learning_rate = 0.000003

for epoch in range(100):    
    z = (x.matmul(w) + b)
    loss = torch.mean((z - y)**2)  
    
    grads = torch.autograd.grad(loss, [w, b])
    
    w.data -= grads[0] * learning_rate
    b.data -= grads[1] * learning_rate

    print("{:3} - loss : {}".format(epoch, loss.item()), end="\r")

#### 3. optimizer 사용하기

In [35]:
opt = torch.optim.SGD([w, b], lr=learning_rate)

In [36]:
for epoch in range(100):    
    z = (x.matmul(w) + b)
    loss = torch.mean((z - y)**2)  
    
    loss.backward()
    opt.step()
    print("{:3} - loss : {}".format(epoch, loss.item()), end="\r")
    
    opt.zero_grad()

In [38]:
y_pred = x.matmul(w) + b
print("예측한 집값 :", y_pred[19].item(), "실제 집값 :", target[19])

예측한 집값 : 21.924237389495104 실제 집값 : 18.2
